#### 전주 데이터 처리

In [1]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
from freeman.plt_setting import plt_settings
from freeman.aiddd.data_manager import read_data, write_data, common_columns

# 차트 한글처리 지원
plt_settings()

In [2]:
_start_time = datetime.now()

# 4번째 설비 갯 수가 추가된 공사비 전처리 데이터 
_df_cons = read_data(
    '1b-merge-cons-device-counts-training', 
    data_type='preprocess', 
    process_seq='4th'
)

# 3번째 제공된 전주 데이터
_df_pole = read_data('POLE_DATA', data_type='provide', process_seq='3rd')

print(
    f'Total Elapsed Time for Data Load: {datetime.now() - _start_time}\n'
    f'Provided Data Shape: {_df_pole.shape}\n'
    f'Preprocessed Data Shape: {_df_cons.shape}'
)

Total Elapsed Time for Data Load: 0:00:17.633953
Provided Data Shape: (38533, 63)
Preprocessed Data Shape: (14728, 18)


In [3]:
df_cons, df_pole = _df_cons.copy(), _df_pole.copy()

#### 데이터 전처리

##### 학습에 필요한 컬럼 추출

In [4]:
# 학습에 필요한 데이터만 추출
training_columns = [
    '공사번호', '전산화번호', 
    '전주종류코드', '전주규격코드', '전주형태코드', 
    'X좌표-Y좌표'
]
df_training = df_pole[training_columns].copy()
df_training.shape

(38533, 6)

##### 컬럼명을 영문으로 변환

In [5]:
# 사용하기 좋게 컬럼명을 영문으로 변경
rename_columns = {
    '공사번호': common_columns['공사번호'],
    '전산화번호': common_columns['전산화번호'], 
    '전주종류코드': common_columns['전주종류코드'],
    '전주규격코드': common_columns['전주규격코드'],
    '전주형태코드': common_columns['전주형태코드'],
    'X좌표-Y좌표': common_columns['X좌표-Y좌표'],
}
df_training.rename(columns=rename_columns, inplace=True)

##### 학습에 필요한 레코드 추출

In [6]:
# 공사비에 있는 공사번호를 포함한 레코드만 학습 대상으로 추출
df_training = df_training[df_training.cons_id.isin(df_cons.cons_id)]
df_training.shape

(26920, 6)

##### 좌표정보 추가

In [7]:
# `coordinate` 컬럼을 이용해 x, y 좌표컬럼 추가
df_training[['x', 'y', 'temp1', 'temp2']] = \
    df_training.coordinate.str.split(',', expand=True)

df_training.drop(columns=['temp1', 'temp2'], inplace=True)
df_training.shape

(26920, 8)

In [8]:
# 컬럼별 데이터 형태 확인
df_training.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26920 entries, 220 to 38532
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cons_id        26920 non-null  object 
 1   comp_id        26920 non-null  object 
 2   pole_type_cd   26825 non-null  object 
 3   pole_spec_cd   26825 non-null  float64
 4   pole_shape_cd  26920 non-null  object 
 5   coordinate     26920 non-null  object 
 6   x              26920 non-null  object 
 7   y              26920 non-null  object 
dtypes: float64(1), object(7)
memory usage: 1.8+ MB


In [9]:
# 좌표값을 문자에서 실수형으로 변환
df_training.x = df_training.x.astype(float)
df_training.y = df_training.y.astype(float)

In [10]:
# `coordinate` 데이터 제거
df_training.drop('coordinate', axis=1, inplace=True)

##### Code데이터 One-Hot Encoding

In [11]:
df_training.pole_type_cd.value_counts()

pole_type_cd
C    24967
H     1435
B      402
1       16
M        4
E        1
Name: count, dtype: int64

In [12]:
df_training.pole_shape_cd.value_counts()

pole_shape_cd
O    26259
G      566
V       95
Name: count, dtype: int64

In [13]:
df_training.pole_spec_cd.value_counts()

pole_spec_cd
10.0    19022
12.0     5591
16.0     1067
14.0      739
6.0       402
11.0        3
8.0         1
Name: count, dtype: int64

In [14]:
# 코드 데이터를 나중에 갯 수로 처리하기 위해 one-hot encoding 처리
df_training = pd.get_dummies(
    df_training, 
    columns=['pole_shape_cd', 'pole_type_cd', 'pole_spec_cd'],
    prefix=['pole_shape', 'pole_type', 'pole_spec']
)

df_training.shape

(26920, 20)

In [15]:
df_training.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26920 entries, 220 to 38532
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cons_id         26920 non-null  object 
 1   comp_id         26920 non-null  object 
 2   x               26920 non-null  float64
 3   y               26920 non-null  float64
 4   pole_shape_G    26920 non-null  bool   
 5   pole_shape_O    26920 non-null  bool   
 6   pole_shape_V    26920 non-null  bool   
 7   pole_type_1     26920 non-null  bool   
 8   pole_type_B     26920 non-null  bool   
 9   pole_type_C     26920 non-null  bool   
 10  pole_type_E     26920 non-null  bool   
 11  pole_type_H     26920 non-null  bool   
 12  pole_type_M     26920 non-null  bool   
 13  pole_spec_6.0   26920 non-null  bool   
 14  pole_spec_8.0   26920 non-null  bool   
 15  pole_spec_10.0  26920 non-null  bool   
 16  pole_spec_11.0  26920 non-null  bool   
 17  pole_spec_12.0  26920 non-null  bo

In [16]:
# 안해도 되지만, bool형 타입 데이터를 보기 좋게 1, 0으로 변환
df_training = df_training.applymap(
    lambda x: int(x) if isinstance(x, bool) else x
)

#### 학습한 전주 데이터 저장
* 이 데이터는 학습에 사용되지 않지만, 이후 작업 처리를 할 경우 불러다 사용하면 됨

In [17]:
write_data('1c-preprocessed-pole', df_training, process_seq='4th')

#### 공사비를 기준으로 한 레코드로 만들기

In [18]:
# 유일값의 공사번호들
unique_cons_ids = df_training.cons_id.unique()
# 해당 공사번호의 레코드 값
cons_id_row_values = []
# 합산할 컬럼들
columns_for_summation = [
    col for col in df_training.columns if col.startswith('pole_')
]

In [19]:
# 공사비의 공사번호 갯 수와 전처리된 전주에 있는 유일한 공사번호 갯 수 비교
df_cons.shape[0], len(unique_cons_ids)

# 공사비에 있는 공사번호 모두에 전주가 포함되어 있음을 알 수 있음.

(14728, 14728)

In [20]:
for cons_id in unique_cons_ids:
    df_temp = df_training[df_training.cons_id == cons_id]
    new_row = df_temp[columns_for_summation].sum().values.tolist()
    cons_id_row_values.append([cons_id] + new_row)

In [21]:
# 그룹 데이터프레임 만들기
df_group = pd.DataFrame(
    cons_id_row_values, columns=['cons_id'] + columns_for_summation
)
df_group.shape

(14728, 17)

In [22]:
# 잘 더해 졌는지 임의 공사번호를 불러와 확인
_CONS_ID = '477420193349'
_df1 = df_training[columns_for_summation][df_training.cons_id==_CONS_ID]
_df2 = df_group[columns_for_summation][df_group.cons_id==_CONS_ID]
_df3 = pd.concat([_df1, _df2], ignore_index=True)
_indexs = _df3.index.tolist()
_indexs[-1] = 'group'
_df3.index = _indexs
_df3

,pole_shape_G,pole_shape_O,pole_shape_V,pole_type_1,pole_type_B,pole_type_C,pole_type_E,pole_type_H,pole_type_M,pole_spec_6.0,pole_spec_8.0,pole_spec_10.0,pole_spec_11.0,pole_spec_12.0,pole_spec_14.0,pole_spec_16.0
0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
5,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
6,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
7,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
group,1,7,0,0,0,4,0,4,0,0,0,1,0,3,2,2


#### 공사비 데이터와 전주 그룹데이터 병합
* 여기서는 아직 공사비별 전주 순서는 처리하지 않음(그 부분은 전선에서 처리)

In [23]:
df_cons_pols_merge = pd.merge(
    df_cons, df_group,
    left_on='cons_id', right_on='cons_id', how='left'
)

In [24]:
write_data('1c-merge-cons-pole', df_cons_pols_merge, process_seq='4th')